In [1]:
from ultralytics import YOLO
import os

In [2]:
data_path = os.path.join(os.getcwd(), 'datasets', 'AblationDataset', 'flow', 'flow.yaml')
test_data_path = os.path.join(os.getcwd(), 'datasets', 'TestDataset', 'flowtest', 'flowtest.yaml')
test_data_path_wt = os.path.join(os.getcwd(), 'datasets', 'TestDataset', 'watertrashtest', 'watertrashtest.yaml')

In [ ]:
model = YOLO('yolo11s-lcbham.yaml')
results = model.train(
    data=data_path,
    epochs=200,
    batch=12,
    imgsz=640,
    name='yolo11s-pt', 
    # optimizer='AdamW',
    # momentum=0.9,
    # lr0=0.02,
    augment=False,
)

100%|██████████| 18.4M/18.4M [00:00<00:00, 58.6MB/s]


New https://pypi.org/project/ultralytics/8.3.105 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/AblationDataset/flow/flow.yaml, epochs=200, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo11s-pt, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False,

train: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/AblationDataset/flow/train/labels.cache... 961 images, 0 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
val: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/AblationDataset/flow/valid/labels.cache... 239 images, 1 backgrounds, 0 corrupt: 100%|██████████| 239/239 [00:00<?, ?it/s]


Plotting labels to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.00046875), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      3.05G      1.899      3.528      1.108          1        640: 100%|██████████| 81/81 [00:07<00:00, 11.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 14.52it/s]

...

    200/200      3.37G     0.9523     0.4479     0.8567          3        640: 100%|██████████| 81/81 [00:06<00:00, 12.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 18.67it/s]

                   all        239        628       0.88      0.865      0.903      0.495



200 epochs completed in 0.411 hours.
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt/weights/last.pt, 19.2MB
Optimizer stripped from /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt/weights/best.pt, 19.2MB

Validating /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:00<00:00, 14.06it/s]


                   all        239        628      0.914      0.848      0.914      0.505
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt


In [4]:
res = model.val(data=test_data_path, conf=0.1)

Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/TestDataset/flowtest/test/labels.cache... 800 images, 1 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:02<00:00, 25.34it/s]


                   all        800       2022      0.843       0.78      0.852       0.45
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt2


In [5]:
res = model.val(data=test_data_path_wt)

Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3080, 9994MiB)


val: Scanning /home/ai-laboratory/Documents/ultralytics-floating-trash/datasets/TestDataset/watertrashtest/test/labels.cache... 2800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2800/2800 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 234/234 [00:07<00:00, 32.86it/s]


                   all       2800      10193      0.432     0.0852      0.256      0.124
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /home/ai-laboratory/Documents/ultralytics-floating-trash/runs/detect/yolo11s-pt3
